# Module 5: Road Sign Detection

While being able to detect AprilTags are very useful, there aren't AprilTags located beneath road signs on actual roads. Therefore, we want to use the concepts of what we applied in Module 4, along with the computer vision techniques we learned in Module 3, to detect the road signs by looking at both their shape and color.

This module should follow Module 4: AprilTags

Run the following block to initialize our <b><span style="color:#154734">JetBot</span></b>.

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import time
import threading
from threading import Event

stop_event = Event()

robot = Robot()
camera = Camera.instance()

### Method

Computer vision and image processing can often be thought of as a pipeline. We take an initial image, then run it through various steps that modify the image until we are able to output the information we want. This is the concept we will implement within this activity.

### Processing Steps

Here are the various steps that we will undertake on our image and the order in which they will be done.

1. HSV Conversion

    As we've previously stated, our images are stored in a BGR format. But this makes it difficult to work with specific shades of colors. Instead, we will use the HSV color space, which stands for Hue, Saturation, and Value.

2. HSV Color Masking

    Now we want to isolate only the color red. The range of values for Hue is between [0, 180], while the range of values for Saturation and Value are between [0, 255]. So we want to find values that work well for detecting the colors of a stop sign.

    However, hue is cyclical, and can be thought of as a circle, where red is at the very edge. That means there are going to be values for the hue of the stop sign that are both small and close to 0, as well as large and close to 180. To make sure we are able to account for all shades of red, we need to include two HSV ranges, that specify a range near 0 and a range near 180.

3. Dilation and Erosion

    Dilation and Erosion are operations that take add and take away pixels respectively, using what is known as a kernel. From our color masking, anything that isn't red was removed and set to 0. Dilation looks at every pixel individually and checks if there is a neighboring pixel that isn't set to 0, and if so, sets the pixel it was checking to a 1. Similarly, erosion looks at a pixel and checks for empty pixels neighboring, and if so, sets that pixel to 0. Doing so for every pixel creates a binary mask that thaat can help remove both small empty spaces and/or small specks.

4. Gaussian Blur

    After the Dilation and Erosion operations have been applied to the image, the gaussian blur evens out the various pixel values of it and its surrounding neighbors. This removes sharp and sudden changes that don't represent true edges and changes between objects in the image.

5. Canny Edge Detection

    This is an algorithm used for detecting edges and contrasts between pixels in an image. In a basic sense, it checks for major differences between the values of pixels. This will return an image of only the edges.

6. Contour Detection and Area Filtering

    With the information of the various edges, we now want to find the various contours or shapes that are seen. Since there are likely many shapes detected by the canny edges, we filter out any that are too small, only looking at bigger shapes, which would include the stop sign.

7. Polygon Approximation

    Now we want to estimate the shape of our remaining shapes, and estimate what they would be if they were polygons. This simplifies the various contours to be compose of vertices and edges, which then allows us to isolate only contours that are made up of 8 vertices, or in other words, represent an octagon.

8. Post-Processing

    Now we have all the information we require, which is applied to our various intermediate steps and returned by the function. This way, we can debug and understand the effects of the various operations in an easier format, and help in determining any issues.

Below is a block that will run and show various intermediate steps to diagnose the best way to detect stop signs. When you run this block, 12 sliders should appear, which will allow you to control the various HSV parameter values. Try tuning them until you are able to achieve a function that is able to accurately detect and identify stop signs within the camera's view.

In [ ]:
# Function of Image Processing
def update_loop():
    while not stop_event.is_set():

        clear_output(wait = True)
        orig_image = np.array(camera.value)
        image = orig_image

        # Convert to HSV (Hue, Saturation, Value) color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # TODO: Change the values of these.
        # Define range of red color in HSV space
        lower_red1 = (h_min1.value, s_min1.value, v_min1.value)
        upper_red1 = (h_max1.value, s_max1.value, v_max1.value)
        lower_red2 = (h_min2.value, s_min2.value, v_min2.value)
        upper_red2 = (h_max2.value, s_max2.value, v_max2.value)

        # Create a mask for red color
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = mask1 | mask2

        # Apply the mask to the image
        red_traffic_signs = cv2.bitwise_and(image, image, mask=mask)

        # Dilate image to get rid of white stop text
        
        dilated_image = cv2.dilate(red_traffic_signs, np.ones((3, 3), np.uint8), iterations = 3)
        eroded_image = cv2.erode(dilated_image, np.ones((5, 5), np.uint8))

        # Convert to grayscale
        gray = cv2.cvtColor(eroded_image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect edges using Canny edge detector
        edges = cv2.Canny(blurred, 100, 200)

        # Find contours
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Minimum area we look at
        min_area = 800

        # Loop through contours
        for contour in contours:
            contour_area = cv2.contourArea(contour)
            if contour_area > min_area:
                # Approximate the contour to a polygon
                epsilon = 0.01 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)

                # If the shape has 8 vertices, it could be a stop sign (octagon)
                if len(approx) == 8:
                    cv2.drawContours(orig_image, [approx], -1, (0, 255, 0), 2)
                    cv2.putText(orig_image, 'Stop Sign', tuple(approx[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    cv2.putText(orig_image, str(contour_area), (approx[0][0][0], approx[0][0][1] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        overlay_widget.value = bgr8_to_jpeg(orig_image)
        red_mask_widget.value = bgr8_to_jpeg(red_traffic_signs)
        dilation_widget.value = bgr8_to_jpeg(dilated_image)
        erosion_widget.value = bgr8_to_jpeg(eroded_image)
        blur_widget.value = bgr8_to_jpeg(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR))
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR))
        time.sleep(0.05)

In [ ]:
# Display Block
# Image Debugging Widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
red_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
dilation_widget = widgets.Image(format='jpeg', width=300, height=300)
erosion_widget = widgets.Image(format='jpeg', width=300, height=300)
blur_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([
    widgets.VBox([widgets.Label("Overlay"), overlay_widget, widgets.Label("Erosion"), erosion_widget]),
    widgets.VBox([widgets.Label("Red Mask"), red_mask_widget, widgets.Label("Gaussian Blur"), blur_widget]),
    widgets.VBox([widgets.Label("Dilation"), dilation_widget, widgets.Label("Canny Edges"), edge_mask_widget])
]))

# Lower Red HSV Sliders
h_min1 = widgets.IntSlider(value=0, min=0, max=180, description='H Min')
s_min1 = widgets.IntSlider(value=100, min=0, max=255, description='S Min')
v_min1 = widgets.IntSlider(value=100, min=0, max=255, description='V Min')
h_max1 = widgets.IntSlider(value=10, min=0, max=180, description='H Max')
s_max1 = widgets.IntSlider(value=255, min=0, max=255, description='S Max')
v_max1 = widgets.IntSlider(value=255, min=0, max=255, description='V Max')
# Upper Red HSV Sliders
h_min2 = widgets.IntSlider(value=169, min=0, max=180, description='H Min')
s_min2 = widgets.IntSlider(value=100, min=0, max=255, description='S Min')
v_min2 = widgets.IntSlider(value=100, min=0, max=255, description='V Min')
h_max2 = widgets.IntSlider(value=180, min=0, max=180, description='H Max')
s_max2 = widgets.IntSlider(value=255, min=0, max=255, description='S Max')
v_max2 = widgets.IntSlider(value=255, min=0, max=255, description='V Max')

display(widgets.VBox([
    widgets.Label("Adjust HSV range for lower red range:"),
    widgets.HBox([h_min1, h_max1]),
    widgets.HBox([s_min1, s_max1]),
    widgets.HBox([v_min1, v_max1]),
    widgets.Label("Adjust HSV range for upper red range:"),
    widgets.HBox([h_min2, h_max2]),
    widgets.HBox([s_min2, s_max2]),
    widgets.HBox([v_min2, v_max2])
]))

thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

Run the block below if you want to try updating anything in the function itself, then run the two blocks above again.

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()